In [1]:
from torch_tmm import Model, BaseLayer, BaseMaterial
from torch_tmm.dispersion import Constant_epsilon, Lorentz, TaucLorentz
import torch
from lmfit import Parameters,Minimizer
import random

In [2]:
random.seed(42)

In [3]:
dtype=torch.complex128
device=torch.device('cpu')

In [4]:
n=10 #number of filter slots on each wheel
N=n**2-1 #Total number if combinations of filters

In [5]:
wavelengths = torch.linspace(8000, 14000, 99) # in [nm]
angles = torch.linspace(0, 89, 90)

<h2> Set up of dispersion functions and materials

In [6]:
#Defining dispersion models
env_disp = [Constant_epsilon(1+0j, dtype, device)]
Metalic_1 = [Constant_epsilon(3+0.2j, dtype=dtype, device=device)]
Metalic_2=[Lorentz(A=0.0015,E0=1/12.5,C=0.014,dtype=dtype,device=device)]
Amorphous=[Lorentz(A=81,E0=1/12.5,C=0.014,dtype=dtype,device=device)]
subs_disp = [Constant_epsilon(5+0j, dtype, device)]

In [7]:
# Defining materials
env_mat = BaseMaterial(env_disp,name = 'air',dtype= dtype,device= device)
Metalic_1_mat = BaseMaterial(Metalic_1,dtype= dtype,device= device)
Metalic_2_mat = BaseMaterial(Metalic_2,dtype= dtype,device= device)
Amorphous_mat = BaseMaterial(Amorphous,dtype= dtype,device= device)
subs_mat = BaseMaterial(subs_disp,name = 'glass',dtype= dtype, device=device)

In [8]:
# Defining layers
env = BaseLayer(env_mat, thickness=0, LayerType='env')
subs = BaseLayer(subs_mat, thickness=0, LayerType='subs')

<h2> Functions used to construct objective function

In [9]:
def generate_single_filter(aSi_thickness, Au_thickness,Ti_thickness):

    """Generate individual filter response.

    Parameters:
    ----------
    aSi_thickness : Pytorch tensor
        Thickness of amorphous silicon

    Au_thickness : Pytorch tensor
        Thickness of gold

    Ti_thickness : Pytorch tensor
        Thickness of titanium

    Returns:
    --------
    Pytorch tensor : Transmission of one filter response

    """

    Ti = BaseLayer(Metalic_1_mat, thickness=Ti_thickness, LayerType='coh')
    Au = BaseLayer(Metalic_2_mat, thickness=Au_thickness, LayerType='coh')
    aSi = BaseLayer(Amorphous_mat, thickness=aSi_thickness, LayerType='coh')

    # Model of the thin film geometry
    model = Model(env, [aSi,Au,Ti], subs, dtype, device)
    results= model.evaluate(wavelengths, angles)

    # Model of the substrate
    model_Si=Model(env,[],subs,dtype,device)
    results_Si=model_Si.evaluate(wavelengths,angles)

    # Transmissions and reflections of the substrate
    Reflection_Si=(results_Si.reflection('s')+results_Si.reflection('p'))/2
    Reflection_Si=Reflection_Si[:,-1]
    Transmission_Si=(results_Si.transmission('s')+results_Si.transmission('p'))/2
    Transmission_Si=Transmission_Si[:,-1]

    # Transmission and reflection of the filter
    Reflection=(results.reflection('s')+results.reflection('p'))/2
    Reflection=Reflection[:,-1]
    Transmission=(results.transmission('s')+results.transmission('p'))/2 # To verification
    Transmission=Transmission[:,-1] # Only taking 90 degrees angle for analysis

    return (Transmission*Transmission_Si)/(1-Reflection*Reflection_Si) # Correction based on the Substrate behaviour

In [10]:
def filter_wheel(params,n):

    """ Generate geometry matrix of filters responses. First nine entries represents individual responses on one wheel and next nine entries represents second wheel. Rest of the entries are combinations of them.

    Parameters:
    ----------
    params : Pythorch tensor
        Consist of thickness values for each layer obtained from params2tensor() function

    n : number of filter wheel slots

    Returns:
    -------
    G_m : Pytorch tensor
        Corresponds to Pytorch tensor which holds all possible combinations of filter responses in format of  N x w

     """


    num_filters = n - 1  # Number of individual filters
    total_transmissions = num_filters * 2  # First and second sets of transmissions
    G_m_list=[]
    transmission_list=[]

    # First set of filters
    for i in range(num_filters):
        filter_response=generate_single_filter(params[36+i],params[18+i],params[i])
        transmission_list.append(filter_response)
        G_m_list.append(filter_response)

    # Second set of filters
    for i in range(num_filters):
        filter_response=generate_single_filter(params[36+num_filters+i],params[18+num_filters+i],params[num_filters+i])
        transmission_list.append(filter_response)
        G_m_list.append(filter_response)

    transmissions = torch.stack(transmission_list)
    running_index=len(G_m_list)

    for i in range(num_filters):  # First 9 transmissions
        for j in range(num_filters, total_transmissions):  # Next 9 transmissions
            if running_index < n ** 2 - 1:
                # Create new combinations without in-place modification of G_m
                combination = transmissions[i] * transmissions[j]
                G_m_list.append(combination)
                running_index += 1

    G_m=torch.stack(G_m_list)

    return G_m

In [11]:
def Dirac(n):

    """ Creates Dirac delta matrix.

    Parameters:
    ----------
        n : number of filter wheel slots

    Returns :
    ---------
    Pytorch tensor
        Matrix contains w x M matrix with Dirac delta functions at different positions
    """

    w = len(wavelengths)  # Number of wavelength points (rows)
    M = n**2 - 1  # Number of Dirac delta positions (columns)

    row_indices = torch.arange(w)
    col_indices = torch.arange(M)

    # Create zeros matrix (leaf tensor with requires_grad=True)
    D_m = torch.zeros(w, M, dtype=torch.float64, requires_grad=True)  # Initially all zeros

    # Use of functional assignment without in-place modification to keep computational graph intact
    D_m = D_m + torch.sparse_coo_tensor(
    indices=torch.stack([row_indices, col_indices]),
    values=torch.ones(row_indices.size(0), dtype=torch.float64),
    size=(w, M)
    ).to_dense()

    return D_m

In [12]:
def params2tensor(params):
    """Convert lmfit.Parameters to PyTorch tensor.

    Parameters:
    ----------
    params : lmfit.Parameters
        Parameter object containing the thickness values for each layer

    Returns:
    ---------
    Pytorch tensor
        All thickness values are converted to the Pytorch tensor and stacked into one matrix with gradient enabled.

    """

    tensor_params = []
    for i in range((n - 1) * 6):
        # Extracting values to Pythorch tensor
        value = params[f't_{i}'].value
        thickness = torch.tensor(value, dtype=torch.float64)
        tensor_params.append(thickness)

    # Stack them into a tensor
    param_tensor = torch.stack(tensor_params, dim=0)
    param_tensor.requires_grad_()  # Set requires_grad at the tensor level
    return param_tensor

<h2>Setting parameters for optimization

In [13]:
params=Parameters()

In [14]:
running_index=0
# Thickness of titanium layer
for i in range((n-1)*2):
    params.add(f't_{running_index}',value=random.uniform(3,10),min=3,max=10)
    running_index+=1

# Thickness of gold layer
for i in range((n-1)*2):
    params.add(f't_{running_index}',value=random.uniform(3,10),min=3,max=10)
    running_index+=1

# Thickness of amorphous silicon layer
for i in range((n-1)*2):
    params.add(f't_{running_index}',value=random.uniform(1000,6000),min=1000,max=6000)
    running_index+=1

<h2> Objective function

In [15]:
def Objective(params):

    """Computes objective function residual.

    Objective function accepts Pytorch tensor and returns a scalar value also as a Pytorch tensor. This method allows for autograd functionality.

    1. Generate geometry matrix
    2. Generate Dirac delta matrix
    3. Calculate coefficient matrix
    4. Approximation of Dirac delta matrix from Coefficient matrix and Geometry matrix
    5. Frobenius norm of residual of Dirac matrices

    Parameters:
    ----------
        params : Pytorch tensor obtained from params2tensor() function

    Returns:
    ---------
        Pytorch tensor
            Scalar value representing Frobenius norm

    """

    # 1. From params -> Geometry matrix
    G_m=filter_wheel(params,n) # Already transposed inside of method

    # 2. Construction of Dirac delta matrix
    D_m=Dirac(n)

    # 3. Coefficient matrix formulation
    C_m=torch.matmul(G_m,D_m)

    # 4. Approximating the Dirac delta matrix
    D_app=torch.linalg.lstsq(G_m,C_m,driver='gelsd')

    # 5. Calculating norm of residual of Dirac matrices
    r=torch.norm(D_app[0]-D_m,p='fro')

    return r

In [16]:
def get_jacobian(params):
    """ Jacobian functional which get as an input params object of lmfit but does not use it to calculate it. It rather takes its value calculated inside of residual() function."""

    global Jacobian, Jaccobian_counter
    Jaccobian_counter+=1
    return Jacobian

In [17]:
def get_hessian(params):
    """ Hessian functional which get as an input params object which is for now I don't know but does not use it anyway  to calculate Hessian. It rather takes its value calculated inside of residual() function."""
    global Hessian, Hessian_counter
    Hessian_counter+=1
    return Hessian

In [18]:
def residuals(params):
    """ Lmfit wrapper of the Pythorch objective function. It takes as input parameters object of lmfit and outputs scalar value of numpy type. All autograd functionality is preserved and calculated inside as well as Jacobian and Hessian.

     Parameters : lmfit.Parameters object

     Returns : numpy scalar value

     """

    param_tensor = params2tensor(params) # Converting to Pytorch tensor
    residual_value=Objective(param_tensor) # Keeps PyTorch graph active

    global Jacobian, Hessian, Residuals_over, optimization_index,Optimization_counter # Global variables used for get_jacobian and get_hessian functional

    # Jacobian and Hessian calculations
    Jacobian=torch.autograd.functional.jacobian(Objective,param_tensor).detach().numpy()
    Hessian=torch.autograd.functional.hessian(Objective,param_tensor).detach().numpy()

    # Detaching residual from computational graph and appending it to history of residuals
    residual_value = residual_value.detach().numpy()
    Residuals_over.append(residual_value)
    Optimization_counter+=1
    optimization_index.append(Optimization_counter)

    return residual_value

<h2> Optimization setup

In [19]:
Jacobian=[]
Hessian=[]

In [20]:
Residuals_over=[]
optimization_index=[]

In [21]:
Jaccobian_counter=0
Hessian_counter=0
Optimization_counter=1

In [22]:
minimizer = Minimizer(residuals, params)
result = minimizer.minimize(method='trust-ncg',jac=get_jacobian,hess=get_hessian)

In [24]:
Jacobian

array([ 8.72891704e-10,  2.31203648e-09,  1.45807973e-10, -1.35598558e-09,
        5.16498199e-11, -1.40276161e-10,  1.24692601e-10,  5.36356972e-10,
       -1.09948430e-10, -1.17788406e-10,  1.24520170e-10, -3.08783437e-10,
        2.52549336e-10, -1.62627657e-10, -4.89460420e-10,  3.16777582e-10,
       -9.46999104e-10, -8.76119810e-10, -5.52296521e-07,  4.28134786e-11,
        8.90303003e-08, -5.40424627e-08, -1.42140312e-07,  2.04569694e-07,
       -2.92835845e-07,  1.48206880e-07, -1.68014125e-07, -3.68588036e-08,
       -3.26749335e-07, -2.69125063e-08, -1.19858086e-07,  2.56510044e-07,
       -1.81269002e-07, -4.45255399e-07,  3.19968885e-07, -1.92867289e-07,
       -1.31244406e-07, -1.00978916e-06, -2.68112618e-07,  7.24806584e-07,
        1.07603153e-08,  1.20129156e-07,  1.40395901e-07, -2.81094319e-07,
        1.85870159e-08,  1.42018430e-08,  2.28111603e-07,  1.34111665e-07,
        2.32282954e-08, -3.09936179e-08,  4.26934218e-07,  1.76748879e-07,
        3.73067810e-07,  

In [25]:
Jaccobian_counter

1

In [26]:
Hessian_counter

1

In [27]:
Optimization_counter

7

In [23]:
result